In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (102 24th St, Miami Beach, FL 33139, USA, (25....
2     (9701 Collins Ave #2303s, Miami Beach, FL 3315...
3     (102 24th St, Miami Beach, FL 33139, USA, (25....
4     (1000 Biscayne Blvd #2201, Miami, FL 33132, US...
5     (FAENA HOUSE, 3315 Collins Ave #7c, Miami Beac...
6     (2821 S Bayshore Dr #11a, Miami, FL 33133, USA...
7     (2669 S Bayshore Dr #1501n, Miami, FL 33133, U...
8     (2333 Fisher Island Dr #4303, Miami Beach, FL ...
9     (18911 Collins Ave APT 1601, Sunny Isles Beach...
10    (16051 Collins Ave APT 2803, Sunny Isles Beach...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,1 Hotel & Homes\r102 24th St PH-1717\rMiami Beach,1/14/2022,4/6/2022,69,"$10,600,000.00",2863,"$3,702.41",Jennifer Restrepo,Alexa Lemieux,1 Hotel & Homes 102 24th St PH-1717 Miami Beach,,1 Hotel & Homes 102 24th St PH-1717 Miami Beach,"(102 24th St, Miami Beach, FL 33139, USA, (25....","(25.7998444, -80.1269823, 0.0)",25.799844,-80.126982,0.0
2,St Regis\r9701 Collins Ave 2303S\rBal Harbour,10/27/2021,4/5/2022,128,"$9,000,000.00",3556,"$2,530.93",Daniel Silverstein,Menachem Fellig,St Regis 9701 Collins Ave 2303S Bal Harbour,St Regis,9701 Collins Ave 2303S Bal Harbour,"(9701 Collins Ave #2303s, Miami Beach, FL 3315...","(25.8884913, -80.1228812, 0.0)",25.888491,-80.122881,0.0
3,1 Hotel & Homes\r102 24th St PH-1706\rMiami Beach,2/24/2022,4/5/2022,8,"$7,500,000.00",2814,"$2,938.87",Tracy Galya,Alberto Esquenazi,1 Hotel & Homes 102 24th St PH-1706 Miami Beach,,1 Hotel & Homes 102 24th St PH-1706 Miami Beach,"(102 24th St, Miami Beach, FL 33139, USA, (25....","(25.7998444, -80.1269823, 0.0)",25.799844,-80.126982,0.0
4,One Thousand Museum\r1000 Biscayne Blvd 2201\r...,1/13/2022,4/4/2022,67,"$6,975,000.00",4600,"$1,516.30",Sonia Toth,Shawn Gamarra,One Thousand Museum 1000 Biscayne Blvd 2201 Miami,One Thousand Museum,1000 Biscayne Blvd 2201 Miami,"(1000 Biscayne Blvd #2201, Miami, FL 33132, US...","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0
5,Faena House\r3315 Collins Ave 7C\rMiami Beach,11/10/2021,4/8/2022,121,"$5,600,000.00",3428,"$2,502.23",Eloy Carmenate,Ethan Pickett,Faena House 3315 Collins Ave 7C Miami Beach,Faena House,3315 Collins Ave 7C Miami Beach,"(FAENA HOUSE, 3315 Collins Ave #7c, Miami Beac...","(25.8078488, -80.1232302, 0.0)",25.807849,-80.123230,0.0
6,Park Grove\r2821 S Bayshore Dr 11A\rMiami,2/17/2022,4/6/2022,18,"$5,200,000.00",3233,"$16,084.13",Ana Prado Contti,Ana Prado Contti,Park Grove 2821 S Bayshore Dr 11A Miami,Park Grove,2821 S Bayshore Dr 11A Miami,"(2821 S Bayshore Dr #11a, Miami, FL 33133, USA...","(25.7283709, -80.238272, 0.0)",25.728371,-80.238272,0.0
7,Grove at Grand Bay\r2669 S Bayshore Dr 1501N\r...,2/18/2022,4/4/2022,15,"$5,025,000.00",3361,"$1,495.09",Cyril Bijaoui,Judith Werner,Grove at Grand Bay 2669 S Bayshore Dr 1501N Miami,Grove at Grand Bay,2669 S Bayshore Dr 1501N Miami,"(2669 S Bayshore Dr #1501n, Miami, FL 33133, U...","(25.7306572, -80.2370979, 0.0)",25.730657,-80.237098,0.0
8,Bayside\r2333 Fisher Island Dr 4303\rMiami Beach,3/7/2022,4/5/2022,12,"$3,590,000.00",1847,"$1,943.69",Robert Vole,Jorge Escasena,Bayside 2333 Fisher Island Dr 4303 Miami Beach,Bayside,2333 Fisher Island Dr 4303 Miami Beach,"(2333 Fisher Island Dr #4303, Miami Beach, FL ...","(25.7589251, -80.143131, 0.0)",25.758925,-80.143131,0.0
9,Ocean Three Condo\r18911 Collins Ave 1601\rSun...,1/27/2022,4/4/2022,48,"$3,550,000.00",3945,$899.87,Daniel Alitenssi,Lana Shvartsburd,Ocean Three Condo 18911 Collins Ave 1601 Sunny...,Ocean Three Condo,18911 Collins Ave 1601 Sunny Isles Beach,"(18911 Collins Ave APT 1601, Sunny Isles Beach...","(25.9509344, -80.11988579999999, 0.0)",25.950934,-80.119886,0.0
10,Turnberry Ocean Colony\r16051 Collins Ave 2803...,3/14/2022,4/8/2022,17,"$3,500,000.00",3480,"$1,005.75",Ilya Reznik,Ilya Reznik,Turnberry Ocean Colony 16051 Collins Ave 2803 ...,Turnberry Ocean Colony,16051 Collins Ave 2803 Sunny Isles Beach,"(16051 Collins Ave APT 2803, Sunny Isles Beach...","(25.9258682, -80.1214311, 0.0)",25.925868,-80.121431,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
#df.at[1,'Buyer Agent']=('Dora Puig, Carl Gambino')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('April 3rd - April 9th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [15]:
m.save('index.html')